# **Digital Signal Processing - Midterm Project**
## **Submitted by:**
| **Figure4 Members** |
|   :----:    |
| *Banta, Ian Noel M.*    |
| *Pabelico, John Paul B.*  |
| *Santander, Mark Adrian P.*   |
| *Villasor, Vince Kazer M.*  |

# **1. Libraries**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# pd.set_option("display.max_rows", None)

# **2. Data**

## 2.1. Extraction

In [2]:
dateDF = pd.read_excel("Figure4_MidtermProject_Data.xlsx", "Sheet1", skiprows=0, nrows= 1350, usecols="A")
humiDF = pd.read_excel("Figure4_MidtermProject_Data.xlsx", "Sheet1", skiprows=0, nrows= 1350, usecols="B")
tempDF = pd.read_excel("Figure4_MidtermProject_Data.xlsx", "Sheet1", skiprows=0, nrows= 1350, usecols="C")

## 2.2. Transformation

### 2.2.1. Data Imputation

In [3]:
humiDF['Humidity (%)'] = humiDF['Humidity (%)'].interpolate()
tempDF['Temperature (°C)'] = tempDF['Temperature (°C)'].interpolate()

## 2.3. Load

In [4]:
datetemphumiDF = pd.DataFrame()
datetemphumiDF = pd.concat([dateDF, humiDF, tempDF], axis=1)

# **3. Functions**

## 3.1. Functions For Section 4.2.1 Time of day

### Filtering Data

In [5]:
def getFilteredData(get, indexesOfTargets, datetemphumiDF):
    indexesOfTargetsPerDay = []
    tempOfTargetsPerDay = []
    humiOfTargetsPerDay = []
    datesOfTargets = []
    minutesOfTargets = []

    tempList = []
    temperatures = []
    humidity = []
    datesList = []
    prevDate = None

    lastIndexFromIndexesOfTargets= indexesOfTargets[-1]
    lastElement = datetemphumiDF['Date'][lastIndexFromIndexesOfTargets]
    
    minutesList = []

    for indx in indexesOfTargets:
        dateTimeSplit = datetemphumiDF['Date'][indx].split('_')
        dateSplit = dateTimeSplit[0].split('/')
        date = dateSplit[1]
        datetimeFromDF = dateDF['Date'][indx]
        timeSplit = dateTimeSplit[1].split(':')
        time = timeSplit[1] + ":" + timeSplit[2]
        
        if (date != prevDate and prevDate != None) or datetimeFromDF == lastElement:
            indexesOfTargetsPerDay.append(tempList)
            tempOfTargetsPerDay.append(temperatures)
            humiOfTargetsPerDay.append(humidity)
            datesOfTargets.append(datesList)
            minutesOfTargets.append(minutesList)
            
            tempList = []
            temperatures = []
            humidity = []
            datesList = []
            minutesList = []
        
        tempList.append(indx)
        temperatures.append(datetemphumiDF['Temperature (°C)'][indx])
        humidity.append(datetemphumiDF['Humidity (%)'][indx])
        datesList.append(datetimeFromDF)
        minutesList.append(time)
        
        prevDate = date
    
    if get == 'temp':
        return zip(tempOfTargetsPerDay, datesOfTargets, minutesOfTargets)
    
    return zip(humiOfTargetsPerDay, datesOfTargets, minutesOfTargets)

### Plotting

In [6]:
def plotFilteredData(show, title, filteredData):
    plt.figure(figsize=(25,5), dpi=500, layout='tight')

    plt.title(title)
    plt.ylabel(show)
    plt.xlabel("DATE AND TIME")

    for item, date, minutes in filteredData:
        dateLabel = date[0].split('/')[1]
        plt.scatter(date, item, label=f'Oct{dateLabel}')

    plt.setp(plt.xticks(fontsize=7))
    plt.gcf().autofmt_xdate()
    plt.legend()
    plt.grid()
    plt.show()

def plotFilteredDataPerDay(show, title, filteredData):
    position = 1
    
    plt.figure(figsize=(30, 25), dpi=500)

    for item, date, minutes in filteredData:
        colorsOfDay = ['blue', 'blue', 'blue', 'blue', 'red', 'red', 'red', 'red']
        
        if len(minutes) < 8:
            colorsOfDay = ['blue', 'blue', 'blue', 'blue', 'red', 'red', 'red']
            
        plt.subplot(4, 4, position)
        plt.scatter(minutes, item, color=colorsOfDay)
        plt.title(date[0].split('_')[0])
        position += 1
    
        plt.ylabel(show)
        plt.xlabel("Minutes")
        plt.grid()
        
    titleSplit = title.split(' ')
    colorsForText = ['blue', 'black', 'red']
    x = 0.43
    
    for word, color in zip(titleSplit, colorsForText):
        plt.text(x, 0.90, word, fontsize=50, color=color, transform=plt.gcf().transFigure)
        x += len(word) / 50
    
    plt.show()

### Central Tendencies

In [8]:
def getMean(x):
    return np.mean(x)

def getMedian(x):
    return np.median(x)

def getMode(x):
    vals, counts = np.unique(x, return_counts=True)
    mode = np.argwhere(counts == np.max(counts))
    return vals[mode].flatten()[0]

def getVariance(x):
    return np.var(x)

def getStandardDeviation(x):
    # standard deviation represents noise and other interference.
    #  the standard deviation is not important in itself, but only in comparison to the mean.
    return np.std(x)

def getSignalToNoiseRatio(mean, std):
    return mean / std

def getCoefficientOfVariation(mean, std):
    #  a signal (or other group of measure values) with a CV of 2%, has an SNR of 50.  Better data means a higher value for the SNR and a lower value for the CV.
    return (std / mean) * 100

def getTargetData(get, indexesOfTargets, dataFrame):
    filteredData = getFilteredData(get, indexesOfTargets, dataFrame)

    targetData = []

    for values, date, minutes in filteredData:
        for item in values:
            targetData.append(item)
    
    return targetData

def getStats(get, indexesOfTargets, dataFrame):
    if get == 'temp':
        temp = getTargetData('temp', indexesOfTargets, dataFrame)

        mean = getMean(temp)
        median = getMedian(temp)
        mode = getMode(temp)
        variance = getVariance(temp)
        standardDeviation = getStandardDeviation(temp)
        snr = getSignalToNoiseRatio(mean, standardDeviation)
        cv = getCoefficientOfVariation(mean, standardDeviation)

        print(f'Mean = {mean}°C')
        print(f'Median = {median}°C')
        print(f'Mode = {mode}°C')
        print(f'Variance = {variance}')
        print(f'Standard Deviation = {standardDeviation}')
        print(f'Signal to Noise Ratio = {snr}')
        print(f'Coefficient of Variation = {cv}%')
    
    else:
        humi = getTargetData('humi', indexesOfTargets, dataFrame)

        mean = getMean(humi)
        median = getMedian(humi)
        mode = getMode(humi)
        variance = getVariance(humi)
        standardDeviation = getStandardDeviation(humi)
        snr = getSignalToNoiseRatio(mean, standardDeviation)
        cv = getCoefficientOfVariation(mean, standardDeviation)

        print(f'Mean = {mean}%')
        print(f'Median = {median}%')
        print(f'Mode = {mode}%')
        print(f'Variance = {variance}')
        print(f'Standard Deviation = {standardDeviation}')
        print(f'Signal to Noise Ratio = {snr}')
        print(f'Coefficient of Variation = {cv}%')

# **4. Analysis Goals:**

## 4.1. What is the periodicity of the data?

FFT haha

In [ ]:
# ahaha

## 4.2. What are the central tendencies of the data according to time of day, day of week, and in general?

### 4.2.1. Time of day

5AM To 6AM

In [ ]:
getStats('temp', indexesOf5AMTo6AM, datetemphumiDF)

In [ ]:
plotFilteredData('Temperature', '5AM TO 6AM', getFilteredData('temp', indexesOf5AMTo6AM, datetemphumiDF))
plotFilteredDataPerDay('Temperature', '5AM TO 6AM', getFilteredData('temp', indexesOf5AMTo6AM, datetemphumiDF))

In [ ]:
getStats('humi', indexesOf5AMTo6AM, datetemphumiDF)

In [ ]:
plotFilteredData('Humidity', '5AM TO 6AM', getFilteredData('humi', indexesOf5AMTo6AM, datetemphumiDF))
plotFilteredDataPerDay('Humidity', '5AM TO 6AM', getFilteredData('humi', indexesOf5AMTo6AM, datetemphumiDF))

1PM To 2PM

In [ ]:
getStats('temp', indexesOf1PMTo2PM, datetemphumiDF)

In [ ]:
plotFilteredData('Temperature', '1PM TO 2PM', getFilteredData('temp', indexesOf1PMTo2PM, datetemphumiDF))
plotFilteredDataPerDay('Temperature', '1PM TO 2PM', getFilteredData('temp', indexesOf1PMTo2PM, datetemphumiDF))

In [ ]:
getStats('humi', indexesOf1PMTo2PM, datetemphumiDF)

In [ ]:
plotFilteredData('Humidity', '1PM TO 2PM', getFilteredData('humi', indexesOf1PMTo2PM, datetemphumiDF))
plotFilteredDataPerDay('Humidity', '1PM TO 2PM', getFilteredData('humi', indexesOf1PMTo2PM, datetemphumiDF))

10PM To 11PM

In [ ]:
getStats('temp', indexesOf10PMTo11PM, datetemphumiDF)

In [ ]:
plotFilteredData('Humidity', '10PM TO 11PM', getFilteredData('temp', indexesOf10PMTo11PM, datetemphumiDF))
plotFilteredDataPerDay('Humidity', '10PM TO 11PM', getFilteredData('temp', indexesOf10PMTo11PM, datetemphumiDF))

In [ ]:
getStats('humi', indexesOf10PMTo11PM, datetemphumiDF)

In [ ]:
plotFilteredData('Humidity', '10PM TO 11PM', getFilteredData('humi', indexesOf10PMTo11PM, datetemphumiDF))
plotFilteredDataPerDay('Humidity', '10PM TO 11PM', getFilteredData('humi', indexesOf10PMTo11PM, datetemphumiDF))

### 4.2.2. Day of week

Mondays To Thursdays

In [ ]:
# haha

Fridays To Sundays

In [ ]:
# haha

### 4.2.3 In general

Ewan ko

In [ ]:
# haha

## 4.3. What patterns arise from the data you are analyzing?

### 4.3.1. idk haha

In [ ]:
# hahaha

# 5. Conclusion?

## 5.1. Haha

### 5.1.1. hahaha